In [2]:
import pandas as pd
import numpy as np
import re, tldextract, time
from sklearn import metrics
import spacy
from spacy import displacy
from collections import Counter

from nltk.tokenize.treebank import TreebankWordDetokenizer
import requests
from thefuzz import process

import time

# get the start time
# from urlextract import URLExtract
# url_extractor = URLExtract()

In [117]:
leader_df  = pd.read_stata('leaders_datapaper_replication_final_9_10_15.dta')
 
leader_df_2 = pd.read_stata('WhyLeadersFightLEADDataset_updated.dta')

country_code = pd.read_csv('country_code.csv')

leader_extra = pd.read_csv('glp_data.csv', sep = ';', header = None)
   
pd.set_option('display.max_columns', 100)


leader_extra.columns = leader_extra.iloc[0]



leader_extra= leader_extra.iloc[1:]

leader_extra['birth_year'] = leader_extra['person_bday'].astype(str).str[0:4]






leader_extra['birth_year'] = leader_extra['birth_year'].astype(float)




C:\Users\casey\AppData\Local\Temp\2\ipykernel_8396\1512014397.py:7: DtypeWarning: Columns (0,4,8,9,12,16,20,29,30,31,32,33,34,35,36,37,38,39,40,41,42,48,49,50,53,66,67,69,70,71,77,78,79,80,81,82,88,89,90,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  leader_extra = pd.read_csv('glp_data.csv', sep = ';', header = None)


In [240]:
conflict_df.keys()



Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp', 'date', 'idacr'],
      dtype='object')

In [194]:
leader_df_code = pd.merge(leader_df, country_code, left_on=['idacr'], right_on = ['alpha-3'], how='inner')



leader_df_narrow = leader_df_code.loc[(leader_df_code['inyear'] >= 1997) ]

leader_df_narrow = leader_df_narrow.dropna(subset=['birthyear'])





In [195]:
name_match = lambda x: process.extractOne(x, leader_extra["person_title"])[2]  # See note below


leader_df['person_abroad'] = leader_extra.loc[leader_df["leadername"].map(name_match).values, ['person_abroad']].values

leader_df['person_college_country'] = leader_extra.loc[leader_df["leadername"].map(name_match).values, ['person_college_country']].values

leader_df['person_eduwest'] = leader_extra.loc[leader_df["leadername"].map(name_match).values, ['person_eduwest']].values



leader_df = leader_df.drop(['year'], axis=1)


KeyboardInterrupt



In [199]:
conflict_df = pd.read_csv('1997-01-01-2023-10-16.csv')



# merge with conflict data
# save this dataframe as a stata data set


In [200]:
from datetime import datetime


date_num = []


for i in conflict_df['event_date']:
    date_extract = datetime.strptime(i, '%d %B %Y')
    date_extract = datetime.strftime(date_extract, '%Y-%m-%d')
    date_num.append(date_extract)



In [201]:
conflict_df['date'] = date_num

# add three digit country code to conflict data
# merge conflict and leader data on three digit code and conflict date falling in between leader term 


idacr = []

for i in conflict_df['event_id_cnty']:
    idacr.append(i[:3])

conflict_df['idacr'] = idacr


conflict_df



,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,actor2,assoc_actor_2,inter2,interaction,civilian_targeting,iso,region,country,admin1,admin2,admin3,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp,date,idacr
0,MEX80068,13 October 2023,2023,1,Political violence,Battles,Armed clash,Unidentified Armed Group (Mexico),NaN,3,Police Forces of Mexico (2018-),NaN,1,13,NaN,484,North America,Mexico,Baja California,Tijuana,NaN,Tijuana,32.5325,-117.0190,1,AFN - Tijuana; Zeta Tijuana,Subnational,"On 13 October 2023, in Tijuana, Baja Californi...",1,NaN,1697477078,2023-10-13,MEX
1,SOM42533,13 October 2023,2023,1,Political violence,Battles,Armed clash,Al Shabaab,NaN,2,Military Forces of Somalia (2022-),NaN,1,12,NaN,706,Eastern Africa,Somalia,Gedo,Baardheere,NaN,Bardera,2.3385,42.2843,1,Calamada,National,"On 13 October 2023, overnight, Al Shabaab carr...",0,NaN,1697481519,2023-10-13,SOM
2,SOM42534,13 October 2023,2023,1,Political violence,Battles,Armed clash,Al Shabaab,NaN,2,Military Forces of Somalia (2022-),NaN,1,12,NaN,706,Eastern Africa,Somalia,Gedo,Luuq,NaN,Luuq,3.8000,42.5446,1,Calamada,National,"On 13 October 2023, overnight, Al Shabaab laun...",0,NaN,1697481519,2023-10-13,SOM
3,SOM42537,13 October 2023,2023,1,Political violence,Battles,Armed clash,Al Shabaab,NaN,2,Military Forces of Somalia (2022-),NaN,1,12,NaN,706,Eastern Africa,Somalia,Lower Shabelle,Marka,NaN,Janaale,1.8000,44.7000,1,Calamada,National,"On 13 October 2023, overnight, Al Shabaab carr...",0,NaN,1697481519,2023-10-13,SOM
4,SUD23109,13 October 2023,2023,1,Political violence,Battles,Armed clash,Military Forces of Sudan (2019-),NaN,1,Rapid Support Forces,NaN,2,12,NaN,729,Northern Africa,Sudan,Khartoum,Karrari,NaN,Al Jarrafa,15.7579,32.5173,1,Sudan Akhbar; Twitter,New media-National,"On 13 October 2023, clashes between SAF and RS...",0,NaN,1697481520,2023-10-13,SUD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255210,SUD2,01 January 1997,1997,1,Political violence,Battles,Armed clash,NDA: National Democratic Alliance (Sudan),NaN,2,Military Forces of Sudan (1989-2019),NaN,1,12,NaN,729,Northern Africa,Sudan,Kassala,Madeinat Kassala,NaN,Kassala,15.4296,36.4105,2,Inter Press Service,International,Fighting between NDA rebels and Sudanese gover...,0,NaN,1653349782,1997-01-01,SUD
255211,UGA1,01 January 1997,1997,3,Political violence,Battles,Armed clash,Military Forces of Uganda (1986-),NaN,1,LRA: Lords Resistance Army,NaN,2,12,NaN,800,Eastern Africa,Uganda,Gulu,Gulu,Pece Division,Gulu,2.7667,32.3056,3,Africa Research Bulletin,Other,Ugandan army battled with LRA rebels - 4 rebel...,4,NaN,1667943770,1997-01-01,UGA
255212,UGA2,01 January 1997,1997,3,Political violence,Battles,Armed clash,Military Forces of Uganda (1986-),NaN,1,UFDF: Uganda Federal Democratic Front,NaN,2,12,NaN,800,Eastern Africa,Uganda,Mityana,Mityana,Central Division,Mityana,0.4015,32.0452,3,Africa Research Bulletin,Other,Over 20 rebel groups believed to belong to Dun...,5,NaN,1667943770,1997-01-01,UGA
255213,SIE2,01 January 1997,1997,3,Political violence,Battles,Government regains territory,Military Forces of Sierra Leone (1996-1997),NaN,1,RUF: Revolutionary United Front,NaN,2,12,NaN,694,Western Africa,Sierra Leone,Eastern,Kono,Gbane,Mandu,8.4642,-10.9332,2,No Peace Without Justice; SL-LED,Local partner-New media,"Around 1 January 1997 (month of), Military For...",0,NaN,1670286851,1997-01-01,SIE


In [196]:
leader_df_narrow

,idacr,ccode,leaderid,leadid29,leadername,birthyear,deathyear,startdate,enddate,inyear,outyear,age,gender,exit,entry,married,marriedinpower,divorced,spousesinlife,sons,daughters,childtotal,adopted,childrendied,onlychild,firstborn,middle,lastborn,firstson,firstdau,parstatus,orphan,orphanbinary,illegit,royalty,dadwork,momwork,kidhealth,primaryedu,boarding,leveledu,milservice,combat,rebel,miledu,warwin,warloss,rebelwin,rebelloss,teacher,journalism,law,engineering,medicine,science,agriculture,religion,labor,activist,careerpolitician,writer,filmmusic,economics,aristocratlandowner,police,militarycareer,interpreter,yrsexper,puppet,physhealth,mentalhealth,scode,xrreg,xrcomp,xropen,xconst,parreg,parcomp,democracy,autocracy,polity,origin,person_abroad,person_college_country,person_eduwest,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
25,USA,2,LEAD.v1-76,A2.9-76,G.W. Bush,1946.0,-777.0,2001-01-21,2004-12-31,2001.0,2004.0,55.0,1,-888.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,Politician,0.0,0.0,1.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,6.0,0.0,0,0,USA,3.0,3.0,4.0,7.0,5.0,5.0,10.0,0.0,10.0,0.0,0,NaN,1,United States of America,US,USA,840,ISO 3166-2:US,Americas,Northern America,NaN,19.0,21.0,NaN
113,DOM,42,LEAD.v1-484,A2.9-484,Hipolito Mejia,1941.0,-777.0,2000-08-17,2004-08-16,2000.0,2004.0,60.0,1,1.0,0.0,1.0,1.0,0.0,1.0,2.0,2.0,4.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0,,0.0,0.0,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,6.0,0.0,0,0,DOM,3.0,3.0,4.0,6.0,2.0,4.0,8.0,0.0,8.0,0.0,NaN,NaN,NaN,Dominican Republic,DO,DOM,214,ISO 3166-2:DO,Americas,Latin America and the Caribbean,Caribbean,19.0,419.0,29.0
144,MEX,70,LEAD.v1-616,A2.9-616,Vicente Fox Quesada,1942.0,-777.0,2000-12-01,2004-12-31,2000.0,2004.0,59.0,1,-888.0,0.0,1.0,1.0,1.0,2.0,2.0,2.0,4.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,Rancher,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12.0,0.0,0,0,MEX,3.0,3.0,4.0,6.0,2.0,4.0,8.0,0.0,8.0,0.0,0,NaN,0,Mexico,MX,MEX,484,ISO 3166-2:MX,Americas,Latin America and the Caribbean,Central America,19.0,419.0,13.0
174,NIC,93,LEAD.v1-1069,A2.9-1069,Aleman,1946.0,-777.0,1997-01-10,2002-01-10,1997.0,2002.0,55.0,1,1.0,0.0,1.0,0.0,0.0,2.0,3.0,4.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,Lawyer,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0,0.0,0,0,NIC,3.0,3.0,4.0,7.0,2.0,3.0,8.0,0.0,8.0,0.0,NaN,NaN,NaN,Nicaragua,NI,NIC,558,ISO 3166-2:NI,Americas,Latin America and the Caribbean,Central America,19.0,419.0,13.0
218,PAN,95,LEAD.v1-1315,A2.9-1315,Mireya Moscoso,1946.0,NaN,1999-09-01,2004-09-01,1999.0,2004.0,55.0,0,1.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0,0,0,0,Teacher,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,5.0,0.0,0,0,PAN,3.0,3.0,4.0,6.0,5.0,5.0,9.0,0.0,9.0,0.0,1.0,NaN,1.0,Panama,PA,PAN,591,ISO 3166-2:PA,Americas,Latin America and the Caribbean,Central America,19.0,419.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,PNG,910,LEAD.v1-8995,A2.9-8995,Morauta,1946.0,-777.0,1999-07-14,2002-08-05,1999.0,2002.0,55.0,1,1.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0,Civil Servant,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,2.0,0.0,0,0,PNG,3.0,3.0,4.0,7.0,5.0,5.0,10.0,0.0,10.0,0.0,0,NaN,0,Papua New Guinea,PG,PNG,598,ISO 3166-2:PG,Oceania,Melanesia,NaN,9.0,54.0,NaN
1566,FJI,950

In [202]:
# Focus on peace project!!

# maybe use narrow leader_df

# change merge so we keep leaders with no conflicts 

# do summary stats for two categories: abroad and not abroad. For each group look at share involved in conflict/length of conflict

# can expand to studied 
df_merged = leader_df_narrow.merge(conflict_df, how='left', on=['idacr']) \
         .query('date.between(`startdate`, `enddate`)')


In [213]:
df_clean = df_merged.merge(leader_df_narrow, on= ['idacr', 'leadername'], how='right', suffixes=('_DROP', '')).filter(regex='^(?!.*_DROP)')

df_clean = df_clean.drop(columns=['region_x'])


In [225]:

pd.set_option('display.max_columns', None)



df_clean['sub-region'].unique()
# write up data description for this dataset



array(['Northern America', 'Latin America and the Caribbean',
       'Western Europe', 'Sub-Saharan Africa',
       'Australia and New Zealand', 'Eastern Europe', 'Southern Europe',
       'Eastern Asia', 'Northern Europe', 'Western Asia', 'Southern Asia',
       'South-eastern Asia', 'Melanesia'], dtype=object)

In [244]:
df_clean['non_west'] = ~df_clean['sub-region'].isin(['Northern America', 'Western Europe', 'Australia and New Zealand', 'Northern Europe' ])*1





In [255]:

df_clean[["year", "idacr", "leadername", "event_type", "person_abroad", "non_west", "person_college_country", "person_eduwest"]]

df_clean[['event_type'].notna()]

AttributeError: 'list' object has no attribute 'notna'

In [215]:
import boto3
import os


from botocore.exceptions import ClientError

import os
from pandasai import PandasAI
from pandasai.llm.openai import OpenAI
from pandasai import SmartDataframe
from langchain.llms import AzureOpenAI


import openai

openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

session = boto3.Session(profile_name='cmoser') # see Amazon_Comprehend_README.md
comp_client = session.client('comprehend')

os.getenv("AZURE_OPENAI_ENDPOINT")

comp_client = session.client('comprehend')


deployment_name='peace-project' 




In [103]:
from selenium import webdriver



from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
url = 'https://www.worldpresidentsdb.com/list/'  # Replace this with the URL of the website you want to scrape
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
text_elements = soup.find_all(['a'])

scraped_text = ' '.join(element.get_text() for element in text_elements)


leader_name_dirty = scraped_text[180:]


leader_name_trans = leader_name_dirty.split('\n\t\t')

leader_name_clean = []
for i in leader_name_trans:
    leader_name_clean.append(i.strip())
    


In [216]:
# Import package
import wikipedia
# Scrape this website https://www.worldpresidentsdb.com/list/
# get names from

# loop wikipedia for "list of heads of government of"

leader_names= leader_list



In [217]:

wiki_list = []

def scrape_wikipedia(leader_names):
    for name in leader_names:
        failed_names= []
        try:
            wiki = wikipedia.page(name, auto_suggest=False)
            text = wiki.content
            wiki_list.append(text)
        except:
            print('name has no page')
            failed_names.append(name)


scrape_wikipedia(leader_names_clean)




name has no page
name has no page


C:\Users\casey\.conda\envs\openai\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\casey\.conda\envs\openai\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page
name has no page


In [218]:

def leader_name(text):
    start_phrase = text + 'Q: Which leader is this text about?' 
    response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=7, temperature = 0)
    text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return text


def country_name(text):
    start_phrase = text + 'Q: What is the name of the country mentioned most? ' 
    response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=7, temperature = 0)
    text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return text


def start_power(text):
    start_phrase = text + 'Q: What year did this leader come into office?' 
    response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=5, temperature =0)
    text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return text




def end_power(text):
    start_phrase = text + "Q: What year did this leader exit office or leave power? A: "
    response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=6, temperature =0)
    text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return text




def lived_abroad(text):
    start_phrase = text + 'Q: Did this leader live abroad before coming into power? Answer this question with either a yes or no' 
    response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=3, temperature =0)
    text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return text


def year_abroad(text):
    start_phrase = text + 'Q: Did this leader live abroad for at least a year before coming into power? Answer this question with either a yes or no' 
    response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=3, temperature =0)
    text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return text


leader_name_list = []
country_name_list= []
start_power_list = []
end_power_list= []
lived_abroad_list = []
year_abroad_list = []

for text in wiki_list:
    if len(text)>80000:
        print('wikipedia too long')
        text_chunk = text[0:8000]
        leader_name_list.append(leader_name(text_chunk))
        country_name_list.append(country_name(text_chunk))
        start_power_list.append(start_power(text_chunk))
        end_power_list.append(end_power(text_chunk))
        lived_abroad_list.append(lived_abroad(text_chunk))
        year_abroad_list.append(year_abroad(text_chunk))
    else:
        # try reading part of text in chunk (1st half)
        #FIX THIS PART OF MY CODE!!
        leader_name_list.append(leader_name(text))
        country_name_list.append(country_name(text))
        start_power_list.append(start_power(text))
        end_power_list.append(end_power(text))
        lived_abroad_list.append(lived_abroad(text))
        year_abroad_list.append(year_abroad(text))
        
   



        



wikipedia too long
wikipedia too long
wikipedia too long
wikipedia too long
wikipedia too long
wikipedia too long
wikipedia too long
wikipedia too long


InvalidRequestError: The response was filtered due to the prompt triggering Azure OpenAI’s content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766

In [ ]:
leader_name_list



In [ ]:

leader_df = pd.DataFrame(
    {'name': leader_name_list,
     'country': country_name_list,
     'start_power': start_power_list,
     'end_power': end_power_list,
     'lived_abroad': lived_abroad_list,
     'year_abroad': year_abroad_list
    })




In [ ]:
leader_df['name'] = leader_df['name'].str.replace(r'A:', '', regex=True)
leader_df['name'] = leader_df['name'].str.replace(r'Q:', '', regex=True)

leader_df['country'] = leader_df['country'].str.replace(r'A:', '', regex=True)
leader_df['country'] = leader_df['country'].str.replace(r'Q:', '', regex=True)

leader_df['start_power'] =leader_df['start_power'].str.replace(r'A:', '', regex=True)
leader_df['end_power'] =leader_df['end_power'].str.replace(r'.', '', regex=True)
leader_df['end_power'] =leader_df['end_power'].str.replace(r'\""', '', regex=True)

leader_df['end_power']= leader_df['end_power'].str.replace(r'A:', '', regex=True)
leader_df['end_power']= leader_df['end_power'].str.replace(r'Q:', '', regex=True)
leader_df['end_power']= leader_df['end_power'].str.replace(r'What', '', regex=True)
leader_df['end_power']= leader_df['end_power'].str.replace(r'Who', '', regex=True)


leader_df['lived_abroad'] = leader_df['lived_abroad'].str.replace(r'A:', '', regex=True)
leader_df['lived_abroad'] = leader_df['lived_abroad'].str.replace(r'Q', '', regex=True)
leader_df['lived_abroad'] = leader_df['lived_abroad'].str.replace(r'.', '', regex=True)
leader_df['year_abroad'] = leader_df['year_abroad'].str.replace(r'.', '', regex=True)
leader_df['year_abroad'] = leader_df['year_abroad'].str.replace(r'A:', '', regex=True)

pat = "(yes)|(no)|(Yes)|(No)"

leader_df['lived_abroad'] = leader_df[['lived_abroad']].apply(lambda r: "/".join(r.str.extractall(pat).stack()), axis=1)
leader_df['year_abroad'] = leader_df[['year_abroad']].apply(lambda r: "/".join(r.str.extractall(pat).stack()), axis=1)


In [ ]:
leader_df
